In [31]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Set paths
image_dir = "segmented_plates"
images = []
labels = []

# Load images and extract full plate labels
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        path = os.path.join(image_dir, filename)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (128, 64))
            img = img / 255.0
            images.append(img)
            label = filename.split(".")[0]  # e.g., P12034
            labels.append(label)

# Convert to arrays
X = np.array(images).reshape(-1, 64, 128, 1)
y = np.array(labels)

print("Loaded", len(X), "images with shape", X.shape)
print("Sample label:", y[0])

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)
y_encoded = to_categorical(encoded_labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,128,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_encoded.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))


Loaded 239 images with shape (239, 64, 128, 1)
Sample label: 100_char0


C:\Users\REIA\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 62, 126, 32)         │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 31, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 29, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 14, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 26880)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │       3,440,768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 239)                 │          30,831 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,490,415 (13.31 MB)

 Trainable params: 3,490,415 (13.31 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - accuracy: 0.0000e+00 - loss: 5.5417 - val_accuracy: 0.0000e+00 - val_loss: 5.5037
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.0000e+00 - loss: 5.4738 - val_accuracy: 0.0000e+00 - val_loss: 5.5015
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 224ms/step - accuracy: 0.0000e+00 - loss: 5.4687 - val_accuracy: 0.0000e+00 - val_loss: 5.5273
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 254ms/step - accuracy: 0.0372 - loss: 5.4511 - val_accuracy: 0.0000e+00 - val_loss: 5.6867
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 246ms/step - accuracy: 0.0000e+00 - loss: 5.4104 - val_accuracy: 0.0000e+00 - val_loss: 5.8349
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 202ms/step - accuracy: 0.0043 - loss: 5.3642 - val_accuracy: 0.0000e+00 - val_loss: 6.1153
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - accuracy: 8.0548e-04 - loss: 5.3067 - val_accuracy: 0.0000e+00 - val_loss: 6.4832
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 246ms/step - acc